In [1]:
import numpy as np
import torch
import time

print("NumPy version: ", np.__version__)
print("PyTorch version: ", torch.__version__)
print("CUDA version: ", torch.version.cuda)

NumPy version:  1.24.3
PyTorch version:  2.0.1
CUDA version:  11.7


THE TWO FIRST ALL SHIFT THE WRONG WAY!!! 

In [3]:
# Empty the CUDA cache to free up GPU memory

In [2]:
def timing_decorator(func):

    """ A decorator that times a function and prints the execution time."""

    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"{func.__name__} took {execution_time:.6f} seconds to run.")
        return result
    
    return wrapper

In [3]:
@timing_decorator
def sort_tensor_by_indices(tensor, primary_index, secondary_index):
    """
    Sort a PyTorch tensor based on both primary and secondary indices.

    Args:
        tensor (torch.Tensor): The input tensor to be sorted.
        primary_index (int): The primary index (column) based on which to sort the tensor. Should be group coulumn
        secondary_index (int): The secondary index (column) based on which to sort the tensor. Should be time column

    Returns:
        torch.Tensor: The sorted tensor.

    Raises:
        ValueError: If the input is not a PyTorch tensor, or if the primary or secondary index is not a natural number.

    """
    # Check if the input is a PyTorch tensor
    if not isinstance(tensor, torch.Tensor):
        raise ValueError("Input 'tensor' must be a PyTorch tensor.")
    
    # Sort based on the secondary index first
    sorted_indices_secondary = torch.argsort(tensor[:, secondary_index])
    tensor_sorted_secondary = tensor[sorted_indices_secondary]

    # Sort based on the primary index
    sorted_indices_primary = torch.argsort(tensor_sorted_secondary[:, primary_index])
    final_sorted_tensor = tensor_sorted_secondary[sorted_indices_primary]

    return final_sorted_tensor


In [4]:
@timing_decorator
def shuffle_tensor_rows(tensor):
    """
    Shuffle the rows of a PyTorch tensor. Just for testing the sorting function.

    Args:
        tensor (torch.Tensor): The input tensor to be shuffled.

    Returns:
        torch.Tensor: The shuffled tensor.

    """
    # Generate random indices for shuffling
    random_indices = torch.randperm(tensor.size(0))

    # Shuffle the tensor using the random indices
    shuffled_tensor = tensor[random_indices]

    return shuffled_tensor

In [87]:
@timing_decorator
def check_if_subset(data, new_data):
    """
    Check if new_data is a subset of the original data by comparing specific columns.

    Args:
        data (torch.Tensor): Original data tensor.
        new_data (torch.Tensor): Data to be checked for subset.

    Returns:
        bool: True if new_data is a subset of the original data; False otherwise.
    """
    
    # Calculate the sum of specific columns
    sum_col = data[:, 2] + data[:, -1]  # group column + target column
    new_sum_col = new_data[:, 2] + new_data[:, -1]  # group column + target column

    # Check if new_sum_col is a subset of sum_col
    is_subset = torch.all(torch.isin(new_sum_col.cpu(), sum_col))

    if is_subset:
        print("The new column is a subset of the original column.")
    else:
        print("The new column is NOT a subset of the original column.")

    return is_subset


In [88]:
@timing_decorator
def check_if_length_correct(data, new_data, steps):
    """
    Check if the number of censored rows in new_data is correct by comparing it with the expected value.

    Args:
        data (torch.Tensor): Original data tensor.
        new_data (torch.Tensor): Data processed by the shift_and_mask_column function.
        steps (int): Number of steps to shift the specified column down.

    Returns:
        bool: True if the number of censored rows in new_data is correct; False otherwise.
    """

    num_groups = torch.unique(data[:, 2]).shape[0]
    num_row_censured = num_groups * abs(steps)

    length_diff = data.shape[0] - new_data.shape[0]

    is_lenght_correct = length_diff == num_row_censured

    if is_lenght_correct:
        print("The number of rows censured is correct.")
    else:
        print("The number of rows censured is incorrect.")

    return is_lenght_correct

In [5]:
@timing_decorator
def generate_synthetic_temporal_dataset(num_groups=3, num_time_steps=5, num_features=1):
    """
    Generate a synthetic temporal dataset with cumulative target values.

    Parameters:
    - num_groups (int): Number of groups.
    - num_time_steps (int): Number of time steps.
    - num_features (int): Number of feature columns.

    Returns:
    - synthetic_dataset (numpy.ndarray): The synthetic dataset with columns for index, time index, group id, features, and target.
    """
    
    n_rows = num_groups * num_time_steps
    
    # Generate the group id column as a repeat pattern
    group_array = np.repeat(np.arange(num_groups), num_time_steps)
    
    # Generate the time index column as a repeat pattern
    time_array = np.tile(np.arange(num_time_steps), num_groups)
    
    # Generate random feature columns
    feature_array = np.random.rand(n_rows, num_features)
    
    # Generate the index column
    indx_array = np.arange(n_rows)
    
    # Initialize the target column with random values
    target_array = np.random.rand(n_rows, 1)
    
    # Combine all columns into a single numpy array
    synthetic_dataset = np.column_stack((indx_array, time_array, group_array, feature_array, target_array))
    
    return synthetic_dataset

# Example usage:
# synthetic_data = generate_synthetic_temporal_dataset(num_groups=3, num_time_steps=5, num_features=1)


# Shifters

In [8]:
# most memory efficient

@timing_decorator
def lead_column_within_groups0(data_org, steps_to_lead=1, column_to_lead=-1, time_column = 1, group_column=2, return_full= False, force_cpu=False):
    """
    Lead (shift down) a specified column within each group in a 2D PyTorch tensor, optionally dropping rows with NaN values induced by the leading operation.
    Note: there should be no NaN values in the specified column before leading!

    Parameters:
    - data_org (torch.Tensor): The original 2D PyTorch tensor. If numpy.ndarray, it will be converted to a PyTorch tensor.
    - column_to_lead (int): The index of the column to be led within each group. This should be the target column.
    - time_column (int): The index of the column that represents the time index.
    - group_column (int): The index of the column that represents the groups.
    - steps_to_lead (int, optional): The number of steps to lead (default is 1).
    - return_full (bool, optional): Whether to return the full data with NaN rows included (default is False). Nice for testing and debugging.
    - force_cpu (bool, optional): Whether to force the data to be on the CPU (default is False). Might be needed for large datasets on small GPUs.

    Returns:
    - data (torch.Tensor): The modified data with the specified column led within each group. If 'return_full' is True, NaN rows are retained.
    """



    # Convert the input NumPy array to a PyTorch tensor if it is not already
    if not isinstance(data_org, torch.Tensor):
        data_org = torch.from_numpy(data_org)

    data = data_org.clone()

    # Move the data to the GPU if available        
    if torch.cuda.is_available() and force_cpu == False:
        data = data.to('cuda')

    # Check that there are no NaN values in the specified column
    if torch.isnan(data[:, column_to_lead]).any():
        raise ValueError("There are NaN values in the specified lead column. Please remove them before leading.")

    # Check that the column to lead is valid
    if steps_to_lead <= 0:
        raise ValueError("Steps to lead must be greater than 0.")
    
    if steps_to_lead > data[:, time_column].max():
        raise ValueError(f"Steps to lead must be less than or equal to the maximum time index which appears to be {int(data[:, time_column].max())}.")

    unique_groups = torch.unique(data[:, group_column])

    for group in unique_groups:
        group_mask = data[:, group_column] == group

        # Create a mask to select rows for the current group
        group_indices = torch.nonzero(group_mask).flatten()

        # Apply leading within the group using slicing
        data[group_indices[steps_to_lead:], column_to_lead] = data[group_indices[:-steps_to_lead], column_to_lead]

        # Fill in the top rows within each group with a desired value (e.g., nan)
        data[group_indices[:steps_to_lead], column_to_lead] = torch.nan

    # This is mostly for testing and debugging purposes
    if return_full == False: 
        nan_mask = ~torch.isnan(data[:, column_to_lead])
        data = data[nan_mask]
    
    # return the modified data with the specified column led within each group. If 'return_full' is True, NaN rows are retained otherwise they are dropped.
    return data


In [9]:
# Fastest

@timing_decorator
def lead_column_within_groups1(data_org, steps_to_lead=1, column_to_lead=-1, time_column = 1, group_column=2, return_full= False, force_cpu=False):
    """
    Lead (shift down) a specified column within each group in a 2D PyTorch tensor, optionally dropping rows with NaN values induced by the leading operation.
    Note: there should be no NaN values in the specified column before leading!

    Parameters:
    - data_org (torch.Tensor): The original 2D PyTorch tensor. If numpy.ndarray, it will be converted to a PyTorch tensor.
    - column_to_lead (int): The index of the column to be led within each group. This should be the target column.
    - time_column (int): The index of the column that represents the time index.
    - group_column (int): The index of the column that represents the groups.
    - steps_to_lead (int, optional): The number of steps to lead (default is 1).
    - return_full (bool, optional): Whether to return the full data with NaN rows included (default is False). Nice for testing and debugging.
    - force_cpu (bool, optional): Whether to force the data to be on the CPU (default is False). Might be needed for large datasets on small GPUs.

    Returns:
    - data (torch.Tensor): The modified data with the specified column led within each group. If 'return_full' is True, NaN rows are retained.
    """

    # Convert the input NumPy array to a PyTorch tensor if it is not already
    if not isinstance(data_org, torch.Tensor):
        data_org = torch.from_numpy(data_org)

    data = data_org.clone()

    # Move the data to the GPU if available        
    if torch.cuda.is_available() and force_cpu == False:
        data = data.to('cuda')

    elif force_cpu == True:
        data = data.clone() 

    # Check that there are no NaN values in the specified column
    if torch.isnan(data[:, column_to_lead]).any():
        raise ValueError("There are NaN values in the specified lead column. Please remove them before leading.")

    # Check that the column to lead is valid
    if steps_to_lead <= 0:
        raise ValueError("Steps to lead must be greater than 0.")
    
    if steps_to_lead > data[:, time_column].max():
        raise ValueError(f"Steps to lead must be less than or equal to the maximum time index which appears to be {int(data[:, time_column].max())}.")


    unique_values, group_indices = data[:, group_column].unique(return_inverse=True)

    # Create a mask for each group
    group_masks = [group_indices == i for i in range(len(unique_values))]

    # Apply the leading operation to the specified column within each group
    for group_mask in group_masks:
        group_rows = data[group_mask]
        nan_tensor = torch.full((steps_to_lead,), float('nan'), dtype=torch.float32).to(data.device)    
        shifted_column = torch.cat((nan_tensor, group_rows[:-steps_to_lead, column_to_lead]), dim=0)
        data[group_mask, column_to_lead] = shifted_column

    # This is mostly for testing and debugging purposes
    if return_full == False: 
        nan_mask = ~torch.isnan(data[:, column_to_lead])
        data = data[nan_mask]
    
    # return the modified data with the specified column led within each group. If 'return_full' is True, NaN rows are retained otherwise they are dropped.
    return data


In [28]:
import torch

@timing_decorator
def shift_and_mask_column(data_org, column_to_shift_idx=-1, time_column_idx=1, steps=-1, force_cpu=False, return_full=False):
    """
    Shifts a specified column down by a given number of steps and replaces specific values with NaN.

    Args:
        data_org (torch.Tensor or numpy.ndarray): Input data tensor. If a numpy array is provided, it will be converted to a torch.Tensor.
        column_to_shift_idx (int, optional): Index of the column to be shifted. Default is -1. I.e., the last column which is usually the target column.
        time_column_idx (int, optional): Index of the time column. Default is 1.
        steps (int, optional): Number of steps to shift the specified column down. Default is -1. I.e. laggging by one step.
        force_cpu (bool, optional): If True, forces computation on CPU. Default is False.
        return_full (bool, optional): Whether to return the full data with NaN rows included (default is False). Nice for testing and debugging.

    Returns:
        torch.Tensor: Processed data tensor. If 'return_full' is True, NaN rows are retained.

    """

    # Convert the input NumPy array to a PyTorch tensor if it is not already
    if not isinstance(data_org, torch.Tensor):
        data_org = torch.from_numpy(data_org)

    data = data_org.clone()

    # Move the data to the GPU if available
    if torch.cuda.is_available() and not force_cpu:
        data = data.to('cuda') # No need to clone since we are moving the data to the GPU
    elif force_cpu:
        data = data.clone() # Need to clone since we are staying on the CPU

    # Extract the column you want to shift
    column_to_shift = data[:, column_to_shift_idx]

    # Create a shifted version of the column
    shifted_column = torch.roll(column_to_shift, shifts=steps, dims=0)  # I could also nan-out stuff before rolling...

    # Create a mask to identify values to be replaced with NaN
    mask = data[:, time_column_idx] < torch.unique(data[:, time_column_idx])[steps] # Need to check that this is robust - and I do not think it works for leading columns
    
    # Create a tensor with NaN values
    nan_tensor = torch.full_like(shifted_column, float('nan'))

    # Apply the mask to replace specific values with NaN
    masked_shifted_column = torch.where(mask, shifted_column, nan_tensor)

    # Replace the original column with the shifted column
    data[:, column_to_shift_idx] = masked_shifted_column

        # This is mostly for testing and debugging purposes
    if return_full == False: 
        nan_mask = ~torch.isnan(data[:, column_to_shift_idx])
        data = data[nan_mask]

    return data


In [111]:
data_org = generate_synthetic_temporal_dataset(num_groups=3, num_time_steps=4, num_features=1) # num_groups would be number of unique pg_id, time would be month_id, num_features would be, well, number of features.

# Convert the input NumPy array to a PyTorch tensor if it is not already
if not isinstance(data_org, torch.Tensor):
    data_org = torch.from_numpy(data_org)

data = data_org.clone()

data

generate_synthetic_temporal_dataset took 0.000360 seconds to run.


tensor([[ 0.0000,  0.0000,  0.0000,  0.7269,  0.0693],
        [ 1.0000,  1.0000,  0.0000,  0.6099,  0.5575],
        [ 2.0000,  2.0000,  0.0000,  0.6151,  0.4719],
        [ 3.0000,  3.0000,  0.0000,  0.9330,  0.3078],
        [ 4.0000,  0.0000,  1.0000,  0.0298,  0.1915],
        [ 5.0000,  1.0000,  1.0000,  0.4825,  0.1551],
        [ 6.0000,  2.0000,  1.0000,  0.0657,  0.3784],
        [ 7.0000,  3.0000,  1.0000,  0.0918,  0.5381],
        [ 8.0000,  0.0000,  2.0000,  0.1473,  0.7178],
        [ 9.0000,  1.0000,  2.0000,  0.7832,  0.0554],
        [10.0000,  2.0000,  2.0000,  0.2622,  0.2436],
        [11.0000,  3.0000,  2.0000,  0.8622,  0.2880]], dtype=torch.float64)

In [112]:
data = shuffle_tensor_rows(data)
data

shuffle_tensor_rows took 0.000652 seconds to run.


tensor([[11.0000,  3.0000,  2.0000,  0.8622,  0.2880],
        [10.0000,  2.0000,  2.0000,  0.2622,  0.2436],
        [ 1.0000,  1.0000,  0.0000,  0.6099,  0.5575],
        [ 2.0000,  2.0000,  0.0000,  0.6151,  0.4719],
        [ 7.0000,  3.0000,  1.0000,  0.0918,  0.5381],
        [ 6.0000,  2.0000,  1.0000,  0.0657,  0.3784],
        [ 5.0000,  1.0000,  1.0000,  0.4825,  0.1551],
        [ 9.0000,  1.0000,  2.0000,  0.7832,  0.0554],
        [ 8.0000,  0.0000,  2.0000,  0.1473,  0.7178],
        [ 3.0000,  3.0000,  0.0000,  0.9330,  0.3078],
        [ 4.0000,  0.0000,  1.0000,  0.0298,  0.1915],
        [ 0.0000,  0.0000,  0.0000,  0.7269,  0.0693]], dtype=torch.float64)

In [116]:
primary_index = 2 # group column
secondary_index = 1 # time column

data = sort_tensor_by_indices(data, primary_index, secondary_index)
data

sort_tensor_by_indices took 0.001090 seconds to run.


tensor([[ 0.0000,  0.0000,  0.0000,  0.7269,  0.0693],
        [ 1.0000,  1.0000,  0.0000,  0.6099,  0.5575],
        [ 2.0000,  2.0000,  0.0000,  0.6151,  0.4719],
        [ 3.0000,  3.0000,  0.0000,  0.9330,  0.3078],
        [ 4.0000,  0.0000,  1.0000,  0.0298,  0.1915],
        [ 5.0000,  1.0000,  1.0000,  0.4825,  0.1551],
        [ 6.0000,  2.0000,  1.0000,  0.0657,  0.3784],
        [ 7.0000,  3.0000,  1.0000,  0.0918,  0.5381],
        [ 8.0000,  0.0000,  2.0000,  0.1473,  0.7178],
        [ 9.0000,  1.0000,  2.0000,  0.7832,  0.0554],
        [10.0000,  2.0000,  2.0000,  0.2622,  0.2436],
        [11.0000,  3.0000,  2.0000,  0.8622,  0.2880]], dtype=torch.float64)

In [118]:
steps_to_shift = -1

new_data = shift_and_mask_column(data, steps=steps_to_shift,return_full=False)

shift_and_mask_column took 0.016423 seconds to run.


In [119]:
check_if_subset(data, new_data)
check_if_length_correct(data, new_data, steps_to_shift)

The new column is a subset of the original column.
check_if_subset took 0.001173 seconds to run.
The number of rows censured is correct.
check_if_length_correct took 0.000104 seconds to run.


True

# OLD!!!!!

In [77]:
new_data

tensor([[ 0.0000,  0.0000,  0.0000,  0.7729,  0.0947],
        [ 1.0000,  1.0000,  0.0000,  0.7822,  0.8289],
        [ 2.0000,  2.0000,  0.0000,  0.4798,  0.4352],
        [ 4.0000,  0.0000,  1.0000,  0.1150,  0.5210],
        [ 5.0000,  1.0000,  1.0000,  0.5573,  0.2652],
        [ 6.0000,  2.0000,  1.0000,  0.0223,  0.2364],
        [ 8.0000,  0.0000,  2.0000,  0.6469,  0.6705],
        [ 9.0000,  1.0000,  2.0000,  0.5260,  0.9028],
        [10.0000,  2.0000,  2.0000,  0.9059,  0.4479]], device='cuda:0',
       dtype=torch.float64)

In [82]:
def check_if_subset(data, new_data):
    """
    Check if new_data is a subset of the original data by comparing specific columns.

    Args:
        data (torch.Tensor): Original data tensor.
        new_data (torch.Tensor): Data to be checked for subset.

    Returns:
        bool: True if new_data is a subset of the original data; False otherwise.
    """
    
    # Calculate the sum of specific columns
    sum_col = data[:, 2] + data[:, -1]  # group column + target column
    new_sum_col = new_data[:, 2] + new_data[:, -1]  # group column + target column

    # Check if new_sum_col is a subset of sum_col
    is_subset = torch.all(torch.isin(new_sum_col.cpu(), sum_col))

    if is_subset:
        print("The new column is a subset of the original column.")
    else:
        print("The new column is NOT a subset of the original column.")

    return is_subset


In [83]:
def check_if_length_correct(data, new_data, steps):
    """
    Check if the number of censored rows in new_data is correct by comparing it with the expected value.

    Args:
        data (torch.Tensor): Original data tensor.
        new_data (torch.Tensor): Data processed by the shift_and_mask_column function.
        steps (int): Number of steps to shift the specified column down.

    Returns:
        bool: True if the number of censored rows in new_data is correct; False otherwise.
    """

    num_groups = torch.unique(data[:, 2]).shape[0]
    num_row_censured = num_groups * abs(steps)

    length_diff = data.shape[0] - new_data.shape[0]

    is_lenght_correct = length_diff == num_row_censured

    if is_lenght_correct:
        print("The number of rows censured is correct.")
    else:
        print("The number of rows censured is incorrect.")

    return is_lenght_correct


In [86]:
check_if_subset(data, new_data)
check_if_length_correct(data, new_data, -1)

The new column is a subset of the original column.
The number of rows censured is correct.


True

In [80]:
data = data
new_data = new_data
steps = -1


def check_if_subset(data, new_data):

    sum_col = data[:,2] + data[:,-1] # group column + target column
    new_sum_col = new_data[:,2] + new_data[:,-1] # group column + target column

    # Check if new_sum_col is a subset of sum_col - it should be if the shift_and_mask_column function worked correctly and no values spilled over to the next group
    is_subset = torch.all(torch.isin(new_sum_col.cpu(), sum_col))

    if is_subset:
        print("The new col is a subset of original col")
    else:
        print("The new col is NOT a subset of original col")


def check_if_lenght_correct(data, new_data, steps):

    num_groups = torch.unique(data[:, 2]).shape[0]
    num_row_censured = num_groups * abs(steps)
    num_row_censured

    length_diff = data.shape[0] - new_data.shape[0]

    if length_diff == num_row_censured:
        print("The number of rows censured is correct.")    

    else:
        print("The number of rows censured is INcorrect.")

In [81]:
check_if_lenght_correct(data, new_data, steps)
check_if_subset(data, new_data)

The number of rows censured is correct.
The new col is a subset of original col


The number of rows censured is correct.


In [63]:
length_diff

3

In [64]:
num_row_censured

6

In [12]:
data_org = generate_synthetic_temporal_dataset(num_groups=15000, num_time_steps=400, num_features=100) # num_groups would be number of unique pg_id, time would be month_id, num_features would be, well, number of features.

generate_synthetic_temporal_dataset took 4.134680 seconds to run.


In [13]:
_ = shift_and_mask_column(data, steps=-1)

shift_and_mask_column took 0.024520 seconds to run.


In [ ]:
def test_small_df(steps_to_lead=1):
    """
    A visial sanity test of the lead_column_within_groups function on a small dataset with 2 groups and 5 time steps and 1 feature column.
    You should see the target column led by "steps_to_lead" steps within each group in the second to last column.
    The original target column is also printed for comparison as the last column. This only happens in this function for testing purposes.
    """
    # Create a small dataset
    small_df = generate_synthetic_temporal_dataset(num_groups=2, num_time_steps=5, num_features=1)

    original_taget = torch.from_numpy(small_df[:, -1][:, np.newaxis])
    if torch.cuda.is_available():
        original_taget = original_taget.to('cuda')

    # Lead the target column within each group by 1 step
   
    small_df0 = lead_column_within_groups0(data_org=small_df, steps_to_lead=steps_to_lead, column_to_lead=-1, time_column = 1, group_column=2, return_full= True)
    small_df0 = torch.cat((small_df0, original_taget), dim=1)

    # Print the results
    print()
    print("Version 0, the most memory efficient")
    print(small_df0)
    print()

    small_df1 = lead_column_within_groups1(data_org=small_df, steps_to_lead=steps_to_lead, column_to_lead=-1, time_column = 1, group_column=2, return_full= True)
    small_df1 = torch.cat((small_df1, original_taget), dim=1)

    print("Version 1, the fastest")
    print(small_df0)        

In [ ]:
test_small_df(steps_to_lead=1)

In [ ]:
data = generate_synthetic_temporal_dataset(num_groups=3000, num_time_steps=400, num_features=100) # num_groups would be number of unique pg_id, time would be month_id, num_features would be, well, number of features.

In [ ]:
new_data = lead_column_within_groups0(data, steps_to_lead = 1, return_full=False) # with 8gb gpu, I cannot handle much more than 10000 groups, 400 time steps and 100 features.

In [ ]:
new_data = lead_column_within_groups1(data, steps_to_lead = 1, return_full=False) # with 8gb gpu, I cannot handle much more than 3000 groups, 400 time steps and 100 features.

In [ ]:
new_data = lead_column_within_groups0(data, steps_to_lead = 1, return_full=False, force_cpu=True)

In [ ]:
new_data = lead_column_within_groups1(data, steps_to_lead = 1, return_full=False, force_cpu=True) 

next solution:

just roll the lead column s steps and then maks it with a nan mask matching the first s rows for each group. Imortant for this solutions that you sort by group first

WORING:

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! LOOPLESS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Needs updated documentation and looots of testing...


@timing_decorator
def shift_and_mask_column(data_org, column_to_shift_idx = -1, time_column_idx = 1, steps=-1, force_cpu=False):
    """
    Shifts a specified column down by a number of steps and replaces specific values with NaN.

    Args:
        data (torch.Tensor): Input data tensor.
        column_to_shift_idx (int): Index of the column to be shifted.
        time_column_idx (int): Index of the time column.
        steps (int, optional): Number of steps to shift the specified column down. Default is -1.

    Returns:
        torch.Tensor: Processed data tensor.

    """

    # Convert the input NumPy array to a PyTorch tensor if it is not already
    if not isinstance(data_org, torch.Tensor):
        data_org = torch.from_numpy(data_org)

    data = data_org.clone()

    # Move the data to the GPU if available        
    if torch.cuda.is_available() and force_cpu == False:
        data = data.to('cuda')

    elif force_cpu == True:
        data = data.clone() 

    # Extract the column you want to shift
    column_to_shift = data[:, column_to_shift_idx]

    # Create a shifted version of the column
    shifted_column = torch.roll(column_to_shift, shifts=steps, dims=0)

    mask = data[:, time_column_idx] < torch.unique(data[:,1])[steps]

    # Create a tensor with NaN values
    nan_tensor = torch.full_like(shifted_column, float('nan'))

    # Apply the mask to replace specific values with NaN
    masked_shifted_column = torch.where(mask, shifted_column, nan_tensor)

    # Replace the original column with the shifted column
    data[:, column_to_shift_idx] = masked_shifted_column

    return data
